In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt

import pynwb
from pynwb import get_class, register_class, load_namespaces
from pynwb.form.utils import docval, getargs, popargs, call_docval_func
from pynwb.file import MultiContainerInterface, NWBContainer, NWBDataInterface
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBAttributeSpec

In [2]:
name = 'multicontainerinterface'
ns_path = name + ".namespace.yaml"
ext_source = name + ".extensions.yaml"

node = NWBGroupSpec(neurodata_type_def='Node',
                    neurodata_type_inc='NWBDataInterface',
                    doc='node in apparatus'
                    )

node_instance = NWBGroupSpec(neurodata_type_inc='Node',
                            doc='node in apparatus', 
                            quantity='*',
                            )

special_node = NWBGroupSpec(neurodata_type_def='SpecialNode',
                            neurodata_type_inc='Node',
                            doc='special node in apparatus', 
                            quantity='*',
                            attributes=[NWBAttributeSpec(name='special',
                                                doc='why the node is special',
                                                dtype='text',
                                                value="just a very cool node")]
                            )

appar = NWBGroupSpec(neurodata_type_def='ApparatusGraph',
                     neurodata_type_inc='NWBDataInterface',
                     name='apparatus_graph',
                     doc='graph of apparatus', 
                     groups=[node_instance],
                    )


ns_builder = NWBNamespaceBuilder(name + ' extensions', name)

ns_builder.add_spec(ext_source, node)
ns_builder.add_spec(ext_source, appar)
ns_builder.add_spec(ext_source, special_node)
ns_builder.export(ns_path)

In [3]:
load_namespaces(ns_path)

@register_class('Node', 'multicontainerinterface')
class Node(NWBDataInterface):

    __nwbfields__ = ('name',)
        
    @docval({'name': 'name', 'type': str, 'doc': 'name of this node'})
    def __init__(self, **kwargs):
        super(Node, self).__init__(name=kwargs['name'])
        
@register_class('SpecialNode', 'multicontainerinterface')
class SpecialNode(Node):

    __nwbfields__ = ('name', 'special')
        
    @docval({'name': 'name', 'type': str, 'doc': 'name of this node'},
            {'name': 'special', 'type': str, 'doc': 'why this node is special'})
    def __init__(self, **kwargs):
        super(Node, self).__init__(name=kwargs['name'])
        self.special = kwargs['special']


@register_class('ApparatusGraph', 'multicontainerinterface')
class ApparatusGraph(MultiContainerInterface):
    """
    Graph of FL_ApparatusNodes connected by undirected FL_ApparatusEdges.
    Represents behaviorally reachable components of an apparatus.
    """
    
    __clsconf__ = {
        'attr': 'my_node',
        'type': NWBDataInterface,
        'add': 'add_node',
        'get': 'get_node',
        'create': 'create_node',
    }

    __help = 'info about ApparatusGraph'

In [4]:
appar = ApparatusGraph(name='hello world')
appar.add_node(SpecialNode(name='FlyingNode', special='this node can fly'))
print(appar)


hello world <class '__main__.ApparatusGraph'>
Fields:
  my_node: { FlyingNode <class '__main__.SpecialNode'> }



In [5]:
import inspect
print(inspect.getcomments(appar.get_node))

None


In [6]:
nwbfile = pynwb.NWBFile("a file", "appar_test", datetime(2018, 6, 1))
pmod = nwbfile.create_processing_module('behavior', 'behavior data and metadata')
pmod.add_container(appar)

with pynwb.NWBHDF5IO('appar_test.nwb', 'w') as io:
    io.write(nwbfile)

/Users/ericmiller/Src/pynwb/src/pynwb/file.py:612: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")
/Users/ericmiller/Src/pynwb/src/pynwb/form/build/map.py:881: MissingRequiredWarning: attribute 'help' for 'apparatus_graph' (ApparatusGraph)
  warnings.warn(msg, MissingRequiredWarning)


In [7]:
io = pynwb.NWBHDF5IO('appar_test.nwb', 'r')
nwb = io.read()
print(nwb.get_processing_module()['behavior'])


apparatus_graph <class '__main__.ApparatusGraph'>
Fields:
  my_node: { }



In [ ]:
help(nwb.get_processing_module()['behavior'])

In [ ]:
io.close()